In [1]:
%reload_ext autoreload
%autoreload 2
from SFM import SFM
import database
import os, shutil
import pycolmap, platform
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
src = "datasets/Viking"
dest = "colmap_test_out"
try:
    os.mkdir(dest)
except:
    shutil.rmtree(dest)
    os.mkdir(dest)

In [3]:
test = SFM(src)
test.ft_extract()
matches = test.ft_match()

templeR_par.txt: 100%|██████████| 49/49 [00:00<00:00, 135.17it/s]


README.txt is not an img...

templeR_ang.txt is not an img...

templeR_par.txt is not an img...



FEATURE EXTRACTION....


Image 46: 100%|██████████| 46/46 [00:03<00:00, 13.86it/s]



MATCHING...



(44, 45): 100%|██████████| 45/45 [00:24<00:00,  1.85it/s]                         

Elapsed Time: 24.33765411376953




In [ ]:
d = test.desc

dd = d[0][1][0].astype(np.int64).flatten()

temp = ' '.join([f"{num}" for num in dd])

f'x y {temp}'

In [ ]:
matches[(1,2)][0]

In [ ]:
# plt.imshow(test.adj_list()[0])
if not test.matches[(10,10)]:
    print("no")

In [ ]:
dir(pycolmap.Database)

In [ ]:
# database.blob_to_array("333333?@",)
# db = pycolmap.Database('testing-database.db')
# db.read_all_cameras()
l = np.random.rand(1000, 2) * (1024, 768)
l.shape

In [4]:
db = database.COLMAPDatabase.connect('testing-database.db')

In [103]:
f = db.execute("""SELECT * FROM matches""")
f.fetchall()

[(1,
  249,
  2,
  b'\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x04\x00\x00\x00\x06\x00\x00\x00\x05\x00\x00\x00\x07\x00\x00\x00\x06\x00\x00\x00\x08\x00\x00\x00\x08\x00\x00\x00\r\x00\x00\x00\x07\x00\x00\x00\x0f\x00\x00\x00\x0e\x00\x00\x00\x11\x00\x00\x00\x0e\x00\x00\x00\x12\x00\x00\x00\x0e\x00\x00\x00\x13\x00\x00\x00\x0c\x00\x00\x00\x15\x00\x00\x00\x11\x00\x00\x00\x17\x00\x00\x00\x15\x00\x00\x00"\x00\x00\x00\x18\x00\x00\x00\'\x00\x00\x00\x1a\x00\x00\x00(\x00\x00\x00\x1c\x00\x00\x00*\x00\x00\x00\x1d\x00\x00\x005\x00\x00\x00%\x00\x00\x00:\x00\x00\x00,\x00\x00\x00<\x00\x00\x005\x00\x00\x00?\x00\x00\x002\x00\x00\x00A\x00\x00\x001\x00\x00\x00F\x00\x00\x006\x00\x00\x00L\x00\x00\x00;\x00\x00\x00M\x00\x00\x00=\x00\x00\x00N\x00\x00\x00I\x00\x00\x00R\x00\x00\x00H\x00\x00\x00U\x00\x00\x00N\x00\x00\x00Z\x00\x00\x00I\x00\x00\x00]\x00\x00\x00V\x00\x00\x00h\x00\x00\x00`\x00\x00\x00i\x00\x00\x00a\x00\x00\x00k\x00\x00\x00f\x00\x00\x00l\x00\x00\x00i\x00\x00\x00m\x00\

In [21]:
# add keypoints

kps_per_frame = [np.array([y.pt for y in list(x)]) for x in test.kp]

for i, frame_kps in enumerate(kps_per_frame):
    db.add_keypoints(i+1, frame_kps)

db.commit()


In [102]:
# add matches
# [(x.queryIdx,x.trainIdx) for x in matches[(0,1)]]
for i in matches.keys():
    if isinstance(i, tuple):
        m = [[x.queryIdx,x.trainIdx] for x in matches[i]]
        if len(m) > 0:
            db.add_matches(i[0], i[1], np.array(m))
db.commit()
# np.random.randint(1000, size=(50, 2))

In [88]:
np.random.randint(1000, size=(50, 2))

array([[461, 449],
       [661, 607],
       [794, 886],
       [350, 162],
       [663, 667],
       [223, 586],
       [562, 555],
       [178, 187],
       [416, 370],
       [684, 104],
       [129, 158],
       [  3,  24],
       [403, 718],
       [959, 986],
       [570, 340],
       [258, 953],
       [ 26, 396],
       [585, 354],
       [626, 898],
       [405, 263],
       [278, 978],
       [ 58,  24],
       [436, 662],
       [106, 574],
       [822,   5],
       [471, 295],
       [757, 350],
       [389, 285],
       [530, 374],
       [621, 243],
       [740, 514],
       [431, 861],
       [182, 221],
       [119,  15],
       [312, 814],
       [274, 945],
       [187, 479],
       [562,  84],
       [880,  52],
       [411, 841],
       [985, 685],
       [856, 139],
       [324, 307],
       [ 24,  23],
       [225, 536],
       [391, 397],
       [996,  44],
       [833, 349],
       [221, 101],
       [391,  60]])